# Step 4: Annual Modeling 
In this notebook, we will try modeling gun violence trends annually. Since we have more annual data than monthly data, our goal is to see whether the annual features will be more helpful here.

In [1]:
# Import defaultdict
from collections import defaultdict

# Numpy and pandas for manipulating the data
import numpy as np
import pandas as pd

# Matplotlib and seaborn for visualization
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns

# GridSearchCV for training 
from sklearn.model_selection import GridSearchCV

# Performance metrics from sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Prophet for time forecasting
from fbprophet import Prophet

# Classification models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# To hide stdout because Prophet can be loud
import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING)

In [2]:
annual_file = './data/cleaned/annual.csv'
by_date_total_file = './data/cleaned/by_date_total.csv'
provisions_file = './data/raw/provisions.csv'
useful_provisions_file = './data/cleaned/useful_provisions.csv'

annual_df = pd.read_csv(annual_file, parse_dates=True, index_col=0)
by_date_total_df = pd.read_csv(by_date_total_file, parse_dates=True, index_col=0)
provisions_df = pd.read_csv(provisions_file, parse_dates=True)
useful_provisions_df = pd.read_csv(useful_provisions_file, parse_dates=True, index_col=0)

## Fixing up features
As before, there are still a couple of features to be tweaked. We need to remove redundant columns and add the label and the provisions.

In [3]:
# Drop District of Columbia
annual_df = annual_df[annual_df['state'] != 'District of Columbia']

# Drop redundant columns
annual_df = annual_df.drop(['gun_deaths_norm', 'other_crime_norm'], axis=1)

# Get states that we will be making models for
states = annual_df['state'].unique()

In [4]:
# Create columns for next year's gun deaths (what we want to predict for each row)
next_annual_df = pd.DataFrame()
next_annual_df['state'] = annual_df['state']
next_annual_df['this_year'] = annual_df['year'] - 1
next_annual_df['next_year'] = annual_df['year']
next_annual_df['next_gun_deaths'] = annual_df['gun_deaths']

# Merge so we have next year's gun deaths in each row
annual_df = pd.merge(annual_df, next_annual_df,
         left_on=['year', 'state'], right_on=['this_year', 'state'])

# Drop year, as it is now named this_year
annual_df = annual_df.drop('year', axis=1)

# Add features from last year in order to view short time trend
features_to_add = annual_df.drop(['this_year', 'next_year',
                                  'state', 'next_gun_deaths'], axis=1).columns

last_annual_df = pd.DataFrame()
last_annual_df['state'] = annual_df['state']
last_annual_df['this_year'] = annual_df['this_year'] + 1
last_annual_df['last_year'] = annual_df['this_year']
for feature in features_to_add:
    last_annual_df['last_' + feature] = annual_df[feature]

annual_df = pd.merge(annual_df, last_annual_df,
                    left_on=['this_year', 'state'], right_on=['this_year', 'state'])

# Replace last year value with change from last year
for feature in features_to_add:
    current = annual_df[feature]
    last = annual_df['last_' + feature]
    annual_df[feature + '_change'] = (current - last) / np.clip(last, 1, None)
    annual_df = annual_df.drop('last_' + feature, axis=1)

In [5]:
# Create a label (gun deaths increases by more than 10%)
rate_change = (annual_df['next_gun_deaths'] - annual_df['gun_deaths'] ) / annual_df['gun_deaths']
annual_df['label'] = rate_change > 0.2
annual_df['label'].sum() # See how many positives we have

157

In [6]:
# Add only the useful provisions to our annual_df (k from this year and k from n years prior)
def add_provisions(annual_df, provisions_df, useful_provisions_df, k=30, n=5):
    # Get the state and year columns for a join later and lawtotal to account for excluded provisions
    columns = list(useful_provisions_df.head(k)['provision'].values)
    columns.extend(['year', 'state', 'lawtotal'])     
    
    # Get the years 
    years = annual_df.groupby('this_year').count().index.values

    # Keep track of provisions for this year and n years prior
    current_provisions = []
    old_provisions = []

    # Add the provisions from each year to a list
    for year in years:
        current_provisions.append(provisions_df[provisions_df['year'] == year][columns])
        old_provisions.append(provisions_df[provisions_df['year'] == year - n][columns])

    # Put the provisions into a DataFrame
    current_provisions = pd.concat(current_provisions)
    old_provisions = pd.concat(old_provisions)
    old_provisions['year'] += n # Match the year which we want to join onto

    # Merge the provisions
    all_provisions = pd.merge(current_provisions, old_provisions, 
                              on=['state', 'year'], suffixes=('', '_old'))

    # Add provisions to annual_df and return the new annual_df
    annual_df = pd.merge(annual_df, all_provisions, 
                          left_on=['this_year', 'state'], 
                          right_on=['year', 'state'])
    return annual_df.drop('year', axis=1)

In [7]:
# Add provisions to our annual_df
annual_df = add_provisions(annual_df, provisions_df, useful_provisions_df)

In [8]:
# Make a function to train our models and return the results
def train_models(annual_df, models, test_year, extra_columns):
    """ Function to train models, returning test and train predictions and trained models.
    feature_df   (DataFrame): Pandas DataFrame with all of the features, including the label
    
    models            (dict): dict with model names as keys and model, params pairs as values 
    
    test_year          (int): Year to test on
    
    extra_columns     (list): List of columns to drop before training (columns that would either 
    not help with the predictions, or would be cheating by using the label itself). 
    
    """
    
    # Initialize the dictionaries we will be returning later
    training_history = defaultdict(list)
    testing_history = defaultdict(list)
    testing_history_probs = defaultdict(list)
    trained_models = defaultdict(dict)
    
    # Train XGBoost for next_year 2000 to 2013; make predictions for 2014-2017
    model, parameters = models['XGBoost']
    
    pretrain_preds = []
    for year in [2014, 2015, 2016, 2017]:
        train_filter = annual_df['next_year'] < year
        test_filter = annual_df['next_year'] == year
    
        X_train = annual_df.dropna(axis=1).loc[train_filter].drop(extra_columns, axis=1).values
        y_train = annual_df.loc[train_filter, 'label']

        X_test = annual_df.dropna(axis=1).loc[test_filter].drop(extra_columns, axis=1).values
        y_test = annual_df.loc[train_filter, 'label']
            
        clf = GridSearchCV(model, parameters)
        clf.fit(X_train, y_train)

        # Make predictions on training set
        test_preds = clf.predict(X_test)
        pretrain_preds.extend(test_preds)

    
    feature_df = annual_df.dropna()
    print(feature_df.shape)
    print(len(pretrain_preds))
    feature_df['pretrain'] = pretrain_preds
    
    # For each state, train a new set of models
    # Training data is all data before next_year
    # Testing data all data during next_year
    train_filter = feature_df['next_year'] < test_year
    test_filter = feature_df['next_year'] >= test_year

    # Partition the feature_df for the training and testing sets
    X_train = feature_df.loc[train_filter].drop(extra_columns, axis=1).values
    y_train = feature_df.loc[train_filter, 'label']

    # Note that the test set has only ONE row for each state. 
    X_test = feature_df.loc[test_filter].drop(extra_columns, axis=1).values
    y_test = feature_df.loc[test_filter, 'label']

    # Keep track of predictions so we can train the meta model as well
    meta_train = []
    meta_test = []
    for name, (model, parameters) in models.items():
        clf = GridSearchCV(model, parameters)
        clf.fit(X_train, y_train)

        # Make predictions on training set
        train_preds = clf.predict(X_train)
        test_preds = clf.predict(X_test)
        train_probs = clf.best_estimator_.predict_proba(X_train)[:, 0]
        test_probs = clf.best_estimator_.predict_proba(X_test)[:, 0]

        # Make meta features to train the meta model on
        meta_train.append(train_probs)
        meta_test.append(test_probs)

        # Keep track of the predictions
        training_history[name].append(train_preds)
        testing_history[name].extend(test_preds)
        testing_history_probs[name].extend(test_probs)

        # Remember the last model
        trained_models[name] = clf

    # Take transpose of meta features so that observations are rows
    meta_train = np.array(meta_train).T
    meta_test = np.array(meta_test).T

    # Create and train the meta model
    clf = GridSearchCV(XGBClassifier(), xgb_params)
    clf.fit(meta_train, y_train)

    # Make training and testing predictions
    train_preds = clf.predict(meta_train)
    test_preds = clf.predict(meta_test)

    # Keep track of the predictions
    training_history['meta'].append(train_preds)
    testing_history['meta'].extend(test_preds)
    testing_history_probs['meta'].extend(test_probs)
    return training_history, testing_history, testing_history_probs, trained_models

## Training models
At this point we should make an important decision. We have to decide the years which we will be training on, as some of our data is not available for earlier years. In this first test, I decided to drop the features that we have insufficient data for, and just training on the features that have complete data for all years.

In [9]:
# Prepare feature_df and extra columns
extra_columns = ['label', 'next_gun_deaths']

# Make dummies for states
annual_df = pd.get_dummies(annual_df, columns=['state'])

In [10]:
# Make our models that we want to train
# Parameters for XGBClassifier
xgb_params = {
  'max_depth': [3, 5, 7, 9], 
  'n_estimators': [30, 50, 100, 300]
}

# Parameters for LogisitcRegression
logi_regr_params = {
    'penalty': ['l1', 'l2'],
    'C': [1e-2, 1e-1, 1, 10, 1e3, 1e5]
}

# Parameters for RandomForest
random_forest_params = {
  'max_depth': [3, 5, 7, 9],
  'n_estimators': [30, 50, 100, 300]
}

# Parameters for AdaBoost
adaboost_params = {
  'n_estimators': [30, 50, 100, 300]
}

knn_params = {
    'n_neighbors': [3, 5, 7],
    'algorithm': ['ball_tree', 'kd_tree']
}

# Parameters for GaussianNB
percent_positive = annual_df['label'].mean() # Percentage of positive labels
percent_negative = 1 - percent_positive # Percentage of negative features 
bayes_params = {'priors': [None, [percent_negative, percent_positive]]}


# Create a dictionary of models with names as keys
# model{ 'model name': (model_object, parameters) } 
models = {
    'XGBoost': (XGBClassifier(), xgb_params), 
    'Logistic Reg': (LogisticRegression(), logi_regr_params),
    'Random Forest': (RandomForestClassifier(), random_forest_params),
    'AdaBoost': (AdaBoostClassifier(), adaboost_params),
    'KNN' : (KNeighborsClassifier(), knn_params),
    'Gaussian NB': (GaussianNB(), bayes_params)
}

In [11]:
test_year = 2016
(training_history, testing_history, 
testing_history_probs, trained_models) = train_models(annual_df, models, test_year, extra_columns)

KeyboardInterrupt: 

In [ ]:
all_preds = []
for v in testing_history.values():
    all_preds.append(v)
    
all_preds = np.array(all_preds).T
vote_by_preds = [int(x > 0.5) for x in all_preds.mean(axis=1)]

testing_history['vote_by_preds'] = vote_by_preds

In [ ]:
date_filter = (annual_df['next_year'] >= 2016)
truth = annual_df.loc[date_filter, 'label']

testing_results_df = pd.DataFrame(annual_df.loc[date_filter, ['label', 'next_date']])
for name, preds in testing_history.items():
    # Add predictions to feature_df
    testing_results_df[name] = preds == truth
    
    # Get accuracy score and confusion matrix
    print("{}: {} ".format(name, accuracy_score(truth, preds)))
    cm = confusion_matrix(truth, preds)
    tn, fp, fn, tp = cm[0][0], cm[0][1], cm[1][0], cm[1][1]
    recall = tp / (tp + fn)
    precision = tp / (tp + fp)
#     print("True Negative: {}".format(tn))
#     print("False Positive: {}".format(fp))
#     print("False Negative: {}".format(fn))
#     print("True Positive: {}".format(tp))
    print("Recall: {}**".format(recall))
    print("Precision: {}".format(precision))
    print(cm)
    print('-'*10)